In [146]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [147]:
titanic_df = pd.read_csv('titanic_processed.csv')

titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,29.00,0,0,7.7750,0,0,1
1,0,3,1,45.50,0,0,7.2250,1,0,0
2,1,1,0,24.00,0,0,69.3000,1,0,0
3,1,1,1,0.92,1,2,151.5500,0,0,1
4,1,1,1,36.00,0,0,26.2875,0,0,1


In [148]:
FEATURES = list(titanic_df.columns[1:])

FEATURES

['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

In [149]:
result_dict = {}

In [150]:
def summarize_classification(y_test, y_pred):
    
    acc = accuracy_score(y_test, y_pred, normalize=True)
    num_acc = accuracy_score(y_test, y_pred, normalize=False)

    prec = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    return {'accuracy': acc, 
            'precision': prec,
            'recall':recall, 
            'accuracy_count':num_acc}

In [151]:
def build_model(classifier_fn,                
                name_of_y_col, 
                names_of_x_cols, 
                dataset, 
                test_frac=0.2):
    
    X = dataset[names_of_x_cols]
    Y = dataset[name_of_y_col]

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_frac)
       
    model = classifier_fn(x_train, y_train)
    
    y_pred = model.predict(x_test)

    y_pred_train = model.predict(x_train)
    
    train_summary = summarize_classification(y_train, y_pred_train)
    test_summary = summarize_classification(y_test, y_pred)
    
    pred_results = pd.DataFrame({'y_test': y_test,
                                 'y_pred': y_pred})
    
    model_crosstab = pd.crosstab(pred_results.y_pred, pred_results.y_test)
    
    return {'training': train_summary, 
            'test': test_summary,
            'confusion_matrix': model_crosstab}

In [152]:
def compare_results():
    for key in result_dict:
        print('Classification: ', key)

        print()
        print('Training data')
        for score in result_dict[key]['training']:
            print(score, result_dict[key]['training'][score])

        print()
        print('Test data')
        for score in result_dict[key]['test']:
            print(score, result_dict[key]['test'][score])
       
        print()

In [153]:
def logistic_fn(x_train, y_train):
    
    model = LogisticRegression(solver='liblinear')
    model.fit(x_train, y_train)
    
    return model

In [154]:
result_dict['survived ~ logistic'] = build_model(logistic_fn,
                                              'Survived',
                                               FEATURES,
                                               titanic_df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7978910369068541
precision 0.7799043062200957
recall 0.7025862068965517
accuracy_count 454

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113



In [155]:
def linear_discriminant_fn(x_train, y_train, solver='svd'): # svd=single value decompostion
    
    model = LinearDiscriminantAnalysis(solver=solver)
    model.fit(x_train, y_train)
    
    return model

In [156]:
# if you run this code and some worrning appear then do this as shown in the next step: FEATURES[0:-1] removing the last column
result_dict['survived ~ linear_discriminant_analysis'] = build_model(linear_discriminant_fn,
                                                                 'Survived',
                                                                  FEATURES,
                                                                  titanic_df)
compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7978910369068541
precision 0.7799043062200957
recall 0.7025862068965517
accuracy_count 454

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7943760984182777
precision 0.7594339622641509
recall 0.7092511013215859
accuracy_count 452

Test data
accuracy 0.7552447552447552
precision 0.7407407407407407
recall 0.6557377049180327
accuracy_count 108



In [157]:
result_dict['survived ~ linear_discriminant_analysis'] = build_model(linear_discriminant_fn,
                                                                     'Survived',
                                                                      FEATURES[0:-1],
                                                                      titanic_df)
compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7978910369068541
precision 0.7799043062200957
recall 0.7025862068965517
accuracy_count 454

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7641509433962265
recall 0.7043478260869566
accuracy_count 451

Test data
accuracy 0.8251748251748252
precision 0.8
recall 0.7586206896551724
accuracy_count 118



In [158]:
def quadratic_discriminant_fn(x_train, y_train):
    
    model = QuadraticDiscriminantAnalysis()
    model.fit(x_train, y_train)
    
    return model

In [159]:
result_dict['survived ~ quadratic_discriminant_analysis'] = build_model(quadratic_discriminant_fn,
                                                                        'Survived',
                                                                        FEATURES[0:-1],
                                                                        titanic_df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7978910369068541
precision 0.7799043062200957
recall 0.7025862068965517
accuracy_count 454

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7641509433962265
recall 0.7043478260869566
accuracy_count 451

Test data
accuracy 0.8251748251748252
precision 0.8
recall 0.7586206896551724
accuracy_count 118

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7702702702702703
recall 0.7184873949579832
accuracy_count 451

Test data
accuracy 0.7902097902097902
precision 0.7083333333333334
recall 0.68
accuracy_count 113



In [160]:
def sgd_fn(x_train, y_train, max_iter=1000, tol=1e-3): # max_iter=1000 if we increase this number will get better result 10000
    
    model = SGDClassifier(max_iter=max_iter, tol=tol)
    model.fit(x_train, y_train)
     
    return model

In [161]:
result_dict['survived ~ sgd'] = build_model(sgd_fn,
                                           'Survived',
                                            FEATURES,
                                            titanic_df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7978910369068541
precision 0.7799043062200957
recall 0.7025862068965517
accuracy_count 454

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7641509433962265
recall 0.7043478260869566
accuracy_count 451

Test data
accuracy 0.8251748251748252
precision 0.8
recall 0.7586206896551724
accuracy_count 118

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7702702702702703
recall 0.7184873949579832
accuracy_count 451

Test data
accuracy 0.7902097902097902
precision 0.7083333333333334
recall 0.68
accuracy_count 113

Classification:  survived ~ sgd

Training data
accuracy 0.6045694200351494
precision 0.5103448275862069
recall 0.9487179487179487
accuracy_count 344

Test data
accuracy 0.49650349650

### LinearSVC

https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html

* SVC with a linear kernel
* dual=False when number of samples > number of features

In [162]:
def linear_svc_fn(x_train, y_train, C=1.0, max_iter=1000, tol=1e-3):
    
    model = LinearSVC(C=C, max_iter=max_iter, tol=tol, dual=False)
    model.fit(x_train, y_train) 
    
    return model

In [163]:
result_dict['survived ~ linear_svc'] = build_model(linear_svc_fn,
                                                  'Survived',
                                                   FEATURES,
                                                   titanic_df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7978910369068541
precision 0.7799043062200957
recall 0.7025862068965517
accuracy_count 454

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7641509433962265
recall 0.7043478260869566
accuracy_count 451

Test data
accuracy 0.8251748251748252
precision 0.8
recall 0.7586206896551724
accuracy_count 118

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7702702702702703
recall 0.7184873949579832
accuracy_count 451

Test data
accuracy 0.7902097902097902
precision 0.7083333333333334
recall 0.68
accuracy_count 113

Classification:  survived ~ sgd

Training data
accuracy 0.6045694200351494
precision 0.5103448275862069
recall 0.9487179487179487
accuracy_count 344

Test data
accuracy 0.49650349650

In [164]:
def radius_neighbor_fn(x_train, y_train, radius=40.0):

    model = RadiusNeighborsClassifier(radius=radius)
    model.fit(x_train, y_train) 
    
    return model

In [165]:
result_dict['survived ~ radius_neighbors'] = build_model(radius_neighbor_fn,
                                                         'Survived',
                                                         FEATURES,
                                                         titanic_df)
compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7978910369068541
precision 0.7799043062200957
recall 0.7025862068965517
accuracy_count 454

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7641509433962265
recall 0.7043478260869566
accuracy_count 451

Test data
accuracy 0.8251748251748252
precision 0.8
recall 0.7586206896551724
accuracy_count 118

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7702702702702703
recall 0.7184873949579832
accuracy_count 451

Test data
accuracy 0.7902097902097902
precision 0.7083333333333334
recall 0.68
accuracy_count 113

Classification:  survived ~ sgd

Training data
accuracy 0.6045694200351494
precision 0.5103448275862069
recall 0.9487179487179487
accuracy_count 344

Test data
accuracy 0.49650349650

max_depth = None [ If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples ]

max_features = None [None -- max_features=n_features, 
                     auto -- then max_features=sqrt(n_features), 
                     sqrt -- then max_features=sqrt(n_features), 
                     log2 -- then max_features=log2(n_features)]

In [166]:
def decision_tree_fn(x_train, y_train, max_depth=None, max_features=None): 
    
    model = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)
    model.fit(x_train, y_train)
    
    return model

In [167]:
result_dict['survived ~ decision_tree'] = build_model(decision_tree_fn,
                                                 'Survived',
                                                  FEATURES,
                                                  titanic_df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7978910369068541
precision 0.7799043062200957
recall 0.7025862068965517
accuracy_count 454

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7641509433962265
recall 0.7043478260869566
accuracy_count 451

Test data
accuracy 0.8251748251748252
precision 0.8
recall 0.7586206896551724
accuracy_count 118

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7702702702702703
recall 0.7184873949579832
accuracy_count 451

Test data
accuracy 0.7902097902097902
precision 0.7083333333333334
recall 0.68
accuracy_count 113

Classification:  survived ~ sgd

Training data
accuracy 0.6045694200351494
precision 0.5103448275862069
recall 0.9487179487179487
accuracy_count 344

Test data
accuracy 0.49650349650

In [168]:
def naive_bayes_fn(x_train,y_train, priors=None):
    
    model = GaussianNB(priors=priors)
    model.fit(x_train, y_train)
    
    return model

In [169]:
result_dict['survived ~ naive_bayes'] = build_model(naive_bayes_fn,
                                                    'Survived',
                                                    FEATURES,
                                                    titanic_df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7978910369068541
precision 0.7799043062200957
recall 0.7025862068965517
accuracy_count 454

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7641509433962265
recall 0.7043478260869566
accuracy_count 451

Test data
accuracy 0.8251748251748252
precision 0.8
recall 0.7586206896551724
accuracy_count 118

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.7926186291739895
precision 0.7702702702702703
recall 0.7184873949579832
accuracy_count 451

Test data
accuracy 0.7902097902097902
precision 0.7083333333333334
recall 0.68
accuracy_count 113

Classification:  survived ~ sgd

Training data
accuracy 0.6045694200351494
precision 0.5103448275862069
recall 0.9487179487179487
accuracy_count 344

Test data
accuracy 0.49650349650